<a href="https://colab.research.google.com/github/alessitomas/paradigma-funcional/blob/main/paradigma_funcional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PARADIGMA FUNCIONAL**

O **paradigma funcional** compreende um estilo de programação com alto nível de abstração, com soluções elegantes, concisas e poderosas. Suas funcões computam um resultado que depende apenas dos valores das entradas, ou seja, não existem **efeitos colaterais** como em programação imperativas.

Por exemplo, vamos considerar a seguinte função (pura) em Python:

**def** soma(x,y):
*   s=x+y
*   **return** s

Observe que este programa realiza um comando imperativo do tipo atribuição, mas não altera nada fora do seu escopo. Se alterasse algo fora do seu escopo, teríamos um efeito colateral no programa.

Na programação funcional tudo é definição de funções (**abstração**) e uso das funções (**aplicação**). No exemplo anterior, teríamos:

1.   **ABSTRAÇÃO**

**def** soma(x,y):
*   s=x+y
*   **return** s

2.   **APLICAÇÃO**

*   res=soma(2,3)

A fundamentação teórica deste paradigma é o Cálculo Lambda ou Cálculo-λ, proposto por Alonzo Church na década de 30, que é um sistema formal para definições, aplicação e recursão de funções.

Entretanto, a primeira linguagem de programação funcional foi Lisp na década de 50, porém atualmente esta já possui muitas características de linguagens imperativas, o que não permite que ela seja considerada uma linguagem funcional pura. Mais recentemente, surgiu uma linguagem puramente funcional, denominada Haskell em homenagem ao lógico Haskell Curry. Dado essa característica, está linguagem que será utilizada neste curso. Além de ser uma linguagem puramente funcional, ela possui diversos recursos avançados de programação, como tipos de dados paramétricos, avaliação preguiçosa, funções de alta ordem e casamento de padrões.

Alguns exemplos de programas funcionais escritos nesta linguagem:


*   lambda a. a
*   lambda a. lambda b. a
*   lambda a. lambda b. b
*   lambda a. (lambda b. (lambda c. a))
*   (lambda true. (lambda false. (lambda true. false)))
*   lambda a. a b
*   (lambda a. a) b
*   (lambda a. a) (b)
*   lambda a. a (b)

A regra para ID é a mesma que seguimos para programas imperativos: começa com uma letra e, depois, podemos ter letras e números.

**O QUE DEVE SER IMPLEMENTADO**

1.   Analisador léxico para reconhecer os tokens desta gramática
2.   Classes para representar os nós da árvore sintática
3.   Analisador sintático para analisar programas funcionais
4.   Visitor para imprimir os nós da árvore sintática





**COMO ENTREGAR**


*   notebook python contendo a implementação
*   depois de cada implementação (analisador léxico, classes da árvore sintática, analisador sintático e PrintVisitor), testes de funcionamento.



In [4]:
!pip install rply

In [22]:
prg = 'int teste = lambda x . x+10'

In [23]:
from rply import LexerGenerator

lg = LexerGenerator()

lg.add('NUMBER', r'\d+')
lg.add('PLUS', r'\+')
lg.add('MINUS', r'-')
lg.add('MUL', r'\*')
lg.add('DIV', r'/')
lg.add('OPEN_PARENS', r'\(')
lg.add('CLOSE_PARENS', r'\)')

lg.add('INT', r'int')
lg.add('STRING', r'string')
lg.add('IF', r'if')
lg.add('ELSE', r'else')
lg.add('WHILE', r'while')
lg.add('FOR', r'for')


lg.add('LAMBDA', r'lambda')
lg.add('DOT', r'\.')

lg.add('ID', r'[a-zA-z][a-zA-z0-9]*')
lg.add('COMP','==')
lg.add('COMP','!=')
lg.add('COMP','>=')
lg.add('COMP','>')
lg.add('COMP','<=')
lg.add('COMP','<')


lg.add('EQUALS', r'=')
lg.add('SEMICOL', r';')

lg.ignore('\s+')

lexer = lg.build()

In [30]:
tokens=lexer.lex(prg)
for token in tokens:
  print(token)

Token('INT', 'int')
Token('ID', 'teste')
Token('EQUALS', '=')
Token('LAMBDA', 'lambda')
Token('ID', 'x')
Token('DOT', '.')
Token('ID', 'x')
Token('PLUS', '+')
Token('NUMBER', '10')


In [25]:
#ÁRVORE SINTÁTICA PREPARADA PARA RECEBER VISITORS

from rply.token import BaseBox

class Prog(BaseBox):
    def __init__(self, decls,stmts):
        self.decls = decls
        self.stmts = stmts

    def accept(self, visitor):
        visitor.visit_prog(self)

class VarDecls(BaseBox):
    def __init__(self, decl,decls):
        self.decl = decl
        self.decls = decls

    def accept(self, visitor):
        visitor.visit_vardecls(self)

class VarDecl(BaseBox):
    def __init__(self, id,tp):
        self.id = id
        self.tp = tp


    def accept(self, visitor):
        visitor.visit_vardecl(self)


class LambdaDecl(BaseBox):
    def __init__(self, id,tp, id2, expr):
        self.id = id
        self.tp = tp
        self.id2 = id2
        self.expr = expr


    def accept(self, visitor):
        visitor.visit_vardecl(self)

class Statements(BaseBox):
    def __init__(self, stmt,stmts):
        self.stmt = stmt
        self.stmts = stmts

    def accept(self, visitor):
        visitor.visit_statements(self)

class Statement(BaseBox):
    def __init__(self,cmd):
        self.cmd = cmd

    def accept(self, visitor):
        visitor.visit_statement(self)

class Atrib(BaseBox):
    def __init__(self, id,expr):
        self.id = id
        self.expr = expr

    def accept(self, visitor):
        visitor.visit_atrib(self)

class IfElse(BaseBox):
    def __init__(self, expr1, comp, expr2, ie1,ie2):
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.ie1=ie1
        self.ie2=ie2

    def accept(self, visitor):
        visitor.visit_ifelse(self)


class While(BaseBox):
    def __init__(self, expr1, comp, expr2, ie1):
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.ie1=ie1


    def accept(self, visitor):
        visitor.visit_while(self)

class For(BaseBox):
    def __init__(self, idinic, exprinic, expr1, comp, expr2, idincr, exprincr, ie1):
        self.idinic=idinic
        self.exprinic=exprinic
        self.expr1=expr1
        self.comp = comp
        self.expr2=expr2
        self.idincr=idincr
        self.exprincr=exprincr
        self.ie1=ie1


    def accept(self, visitor):
        visitor.visit_for(self)

class Expr(BaseBox):
    def accept(self, visitor):
        method_name = 'visit_{}'.format(self.__class__.__name__.lower())
        visit = getattr(visitor, method_name)
        visit(self)

class Id(Expr):
    def __init__(self, value):
        self.value = value

class Number(Expr):
    def __init__(self, value):
        self.value = value


class BinaryOp(Expr):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOp):
  pass


class Sub(BinaryOp):
  pass


class Mul(BinaryOp):
  pass


class Div(BinaryOp):
  pass


In [31]:
#ANALISADOR SINTÁTICO

from rply import ParserGenerator

pg = ParserGenerator(
    # A list of all token names, accepted by the lexer.
    ['NUMBER', 'OPEN_PARENS', 'CLOSE_PARENS',
     'PLUS', 'MINUS', 'MUL', 'DIV', 'INT', 'STRING', 'ID','SEMICOL',
     'EQUALS','COMP','LAMBDA','DOT','IF','ELSE','WHILE','FOR'
    ],
    # A list of precedence rules with ascending precedence, to
    # disambiguate ambiguous production rules.
    precedence=[
        ('left', ['PLUS', 'MINUS']),
        ('left', ['MUL', 'DIV'])
    ]
)

@pg.production('prog : vardecls statements')
def prog(p):
    return Prog(p[0],p[1])

##################################################
# DECLARAÇÕES DE VARIÁVEIS
##################################################

@pg.production('vardecls : vardecl')
def vardecls(p):
    return VarDecls(p[0],None)

@pg.production('vardecls : vardecl vardecls')
def vardecls(p):
    return VarDecls(p[0],p[1])

@pg.production('vardecl : STRING ID SEMICOL')
def vardecl_string(p):
    return VarDecl(p[1].getstr(), "string")

@pg.production('vardecl : INT ID SEMICOL')
def vardecl_int(p):
    return VarDecl(p[1].getstr(), "int")


##################################################
# DECLARAÇÕES DE LAMBDA
##################################################

@pg.production('vardecl : INT ID EQUALS LAMBDA ID DOT expression SEMICOL')
def lambdadecl(p):
    return LambdaDecl(p[1].getstr(), "int", p[4].getstr(), p[6])

##################################################
# COMANDOS - CASO ABERTO
##################################################

@pg.production('statements : openstatement')
def statement_statements(p):
    return Statements(p[0],None)

@pg.production('statements : openstatement statements')
def statement_statements(p):
    return Statements(p[0],p[1])

@pg.production('openstatement : ID EQUALS expression SEMICOL')
def statement_atrib(p):
    return Atrib(p[0].getstr(),p[2])

@pg.production('openstatement : IF OPEN_PARENS expression COMP expression CLOSE_PARENS openstatement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[6],None)


@pg.production('openstatement : IF OPEN_PARENS expression COMP expression CLOSE_PARENS closedstatement ELSE openstatement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[6],p[8])

@pg.production('openstatement : WHILE OPEN_PARENS expression COMP expression CLOSE_PARENS openstatement')
def statement_while(p):
    return While (p[2],p[3],p[4],p[6])

@pg.production('openstatement : FOR OPEN_PARENS ID EQUALS expression SEMICOL expression COMP expression SEMICOL ID EQUALS expression CLOSE_PARENS openstatement')
def statement_while(p):
    return For (p[2].getstr(),p[4],p[6],p[7],p[8],p[10].getstr(),p[12],p[14])


##################################################
# COMANDOS - CASO FECHADO
##################################################

@pg.production('closedstatement : ID EQUALS expression SEMICOL')
def statement_atrib(p):
    return Atrib(p[0].getstr(),p[2])

@pg.production('closedstatement : IF OPEN_PARENS expression COMP expression CLOSE_PARENS closedstatement ELSE closedstatement')
def expression_ifelse1(p):
    return IfElse (p[2],p[3],p[4],p[6],p[8])

@pg.production('closedstatement : WHILE OPEN_PARENS expression COMP expression CLOSE_PARENS closedstatement')
def statement_while(p):
    return While (p[2],p[3],p[4],p[6])

@pg.production('closedstatement : FOR OPEN_PARENS ID EQUALS expression SEMICOL expression COMP expression SEMICOL ID EQUALS expression CLOSE_PARENS closedstatement')
def statement_while(p):
    return For (p[2].getstr(),p[4],p[6],p[7],p[8],p[10].getstr(),p[12],p[14])

@pg.production('expression : ID')
def expression_id(p):
    return Id(p[0].getstr())

@pg.production('expression : NUMBER')
def expression_number(p):
    return Number(int(p[0].getstr()))

@pg.production('expression : OPEN_PARENS expression CLOSE_PARENS')
def expression_parens(p):
    return p[1]

@pg.production('expression : expression PLUS expression')
@pg.production('expression : expression MINUS expression')
@pg.production('expression : expression MUL expression')
@pg.production('expression : expression DIV expression')
def expression_binop(p):
    left = p[0]
    right = p[2]
    if p[1].gettokentype() == 'PLUS':
        return Add(left, right)
    elif p[1].gettokentype() == 'MINUS':
        return Sub(left, right)
    elif p[1].gettokentype() == 'MUL':
        return Mul(left, right)
    elif p[1].gettokentype() == 'DIV':
        return Div(left, right)
    else:
        raise AssertionError('Oops, this should not be possible!')

parser = pg.build()

In [32]:
arvore=parser.parse(lexer.lex(prg))

ParsingError: ignored